In [12]:
import pandas as pd
pd.set_option('display.max_rows', None)

from catlearn.featurize.setup import FeatureGenerator
from catlearn.regression import GaussianProcess
from catlearn.regression.cost_function import get_error
from catlearn.fingerprint.voro import VoronoiFingerprintGenerator

In [10]:
y_targets = pd.read_csv("Y_targerts.csv", header=None)

x_features = pd.read_csv("X_features.csv", header=None)

test_features = pd.read_csv("test_features_2.csv", header=None)
# test_features = test_features.drop([0], axis=1)

In [16]:
test_features

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,0,3.221827,-1.321248,-1.750212,0.082572,0.650287,1.372921,0.817802,0.119196,0.791258,...,0.150663,0.068038,0.600664,0.328624,-0.100715,0.218603,0.254477,0.283785,-0.110971,-0.047266
1,100,-14.031542,-11.221403,6.445213,-6.439186,0.071835,0.669910,-0.205776,0.727237,-0.560234,...,-0.609641,-0.676416,0.089733,-0.061856,0.202401,0.028033,0.316443,-0.137287,0.074904,0.067097
2,101,-0.430953,9.086225,0.603108,-1.856152,-0.299472,-0.325892,0.272450,-0.252936,-0.025560,...,-0.324260,-0.136200,0.025980,0.239057,0.069052,0.008489,-0.037599,0.027642,0.040572,0.004756
3,102,1.570630,2.946260,-2.290912,0.993377,-0.331793,-0.919761,-0.830839,0.380641,-0.719391,...,-0.217211,-0.071917,0.014808,0.048271,-0.018376,-0.002492,-0.028174,-0.063423,-0.014770,-0.004449
4,103,0.173271,-1.775773,7.180974,-4.226729,2.077040,1.881833,-3.334077,-0.298415,0.611100,...,-0.180843,0.204967,0.703092,0.207090,-0.418901,0.011436,-0.120475,-0.115103,-0.051226,-0.069175
5,104,1.714051,4.005601,4.538596,-2.291446,-0.479080,0.140331,0.379073,-0.948423,1.058257,...,0.324604,-0.537254,0.038235,0.336961,-0.059801,0.044210,-0.021405,-0.077450,-0.067374,-0.000314
6,105,-15.669720,-0.638203,-0.881582,-1.799686,0.802989,0.415294,-0.264812,1.048902,-0.219125,...,0.804053,-0.390995,0.048379,0.521057,-0.127417,-0.107659,-0.102565,0.024690,0.092914,0.002848
7,106,-8.772373,7.038291,-0.883081,1.405734,2.053851,-0.737476,0.764161,-0.416956,-0.613754,...,0.425611,0.325503,0.226205,0.055715,-0.222731,0.089304,-0.102651,-0.060030,0.186126,0.182095
8,107,-0.756497,2.433970,4.611855,-2.853741,-0.431183,0.176511,0.236267,-0.668937,0.789545,...,0.210502,-0.552983,0.023831,0.297996,-0.033363,0.018834,-0.018129,-0.076899,-0.037182,0.002398
9,108,1.585182,2.771328,-2.405955,0.973664,-0.275959,-1.001701,-0.762929,0.437257,-0.685592,...,-0.222726,-0.107227,0.024753,0.030408,-0.024788,-0.008040,-0.021035,-0.069463,-0.015086,-0.006849


In [18]:
308 - 39 - 10

259

In [21]:
len(test_features[:259])

259

In [ ]:
test_features = test_features[:259]

In [3]:
len(list(test_features))

20

In [6]:
sigma_f = 1.
w1 = 1.0  # Too large widths results in a biased model.
scaling = 1.0
# width_bounds = ((0.01, 20.0),) * len(train_features[0])
scaling_bounds = ((sigma_f, sigma_f),)
reg_bounds = (1e-1, 1e-1)

kdict = [{
    'type': 'gaussian',
    'width': w1,
    'scaling': sigma_f,
    'dimension': 'features',
    }]

gp = GaussianProcess(
    train_fp=x_features.as_matrix(),
    train_target=y_targets[0].tolist(),
    kernel_list=kdict,
#     gradients=None,

        regularization=1e-2,  # Needs to be defined
#     regularization=1e-1,
#         regularization_bounds=reg_bounds,

#     optimize_hyperparameters=True,
#     scale_optimizer=False,
#     scale_data=False,
    )

# gp.optimize_hyperparameters(global_opt=True, eval_jac=False)

pred = gp.predict(
    test_fp=test_features.as_matrix(),
    uncertainty=True,
    )

/home/raul_surface/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()
/home/raul_surface/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:33: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [ ]:
# data_final = []
# for i in pred_list:
#     df_i = pd.DataFrame(i)
#     df_i["ind"] = [i for i in range(len(df_i))]

#     df_tmp_i = df_i.sort_values("prediction")
#     df_tmp_i["ind_new"] = [i for i in range(len(df_tmp_i))]

#     df_tmp_i["ind_old"] = df_tmp_i["ind"]
#     df_tmp_i["ind"] = df_tmp_i["ind_new"]

# data1 = process_data_for_plot(
#     df_tmp_i,
#     color0=random.choice(color_list),
#     color1="black",
#     name=str(df_i["num_training"].iloc[0]),
#     energy_key="prediction",
#     uncertainty_key="uncertainty",
#     )
# data_final += data1

In [ ]:
layout = get_layout()
layout["yaxis"]["range"] = [-1., 1.]

layout["showlegend"] = True
fig = dict(data=data_final, layout=layout)
py.iplot(fig,
    filename=os.path.join(
        save_dir,
        "ml_regression_test",
        "190317_iro3_ml_training",
        ),
    )

In [ ]:
# for i_cnt, row_i in x_features.iterrows():
#     tmp = 42